# Neural Networks for Empirical Asset Pricing

**PyTorch Implementation**

This notebook demonstrates training deep neural networks to predict asset returns based on firm characteristics.

## Citation
Gu, S., Kelly, B., & Xiu, D. (2020). "Empirical Asset Pricing via Machine Learning."  
*Review of Financial Studies*, 33(5), 2223-2273.

## Overview
- **Models**: 5 feedforward neural network architectures (NN1-NN5) with varying depths
- **Features**: Firm characteristics and their macro interactions
- **Training**: Ensemble learning with hyperparameter grid search
- **Evaluation**: In-sample and out-of-sample R²

## Part 0: Setup and Installation

### Install Required Packages

This implementation uses **PyTorch** instead of TensorFlow/Keras.

Run the following to install PyTorch:

In [ ]:
# Install PyTorch (CPU version)
# For GPU version, see: https://pytorch.org/get-started/locally/
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Install other dependencies
!pip install numpy pandas matplotlib

### Import Libraries

Verify all packages are installed correctly:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Part 1: Generate Simulation Data

First, we generate synthetic asset pricing data using the DGP (Data Generating Process).

The DGP creates:
- **Firm characteristics**: Persistent (AR1) features normalized to [-1, 1]
- **Returns**: Both linear and nonlinear functions of characteristics
- **Factor structure**: Systematic risk factors
- **Macro interactions**: Characteristic effects varying with business cycle

In [ ]:
from DGP import dgp

# Generate data with 100 characteristics for 1-month horizon
print("Generating simulation data...")
print("This will create 100 Monte Carlo samples")
print("Each sample has:")
print("  - 200 firms")
print("  - 180 months")
print("  - 100 characteristics (200 features with macro interactions)")
print("\nEstimated time: 2-3 minutes\n")

dgp(datanum=100, hh=[1])

print("\n" + "="*50)
print("Data generation complete!")
print("Files saved to: ./Simu/SimuData_100/")
print("="*50)

## Part 2: Verify Generated Data

Let's inspect the generated data to understand its structure:

In [ ]:
import os

# Check files
data_dir = './Simu/SimuData_100/'
files = os.listdir(data_dir)
print(f"Number of files: {len(files)}")
print(f"\nExample files:")
for f in sorted(files)[:5]:
    print(f"  {f}")

# Load and inspect first Monte Carlo sample
print("\n" + "="*50)
print("Inspecting Monte Carlo sample 1:")
print("="*50)

# Characteristics
c = pd.read_csv(f'{data_dir}/c1.csv')
print(f"\nCharacteristics shape: {c.shape}")
print(f"  - {c.shape[0]} observations (200 firms × 180 months)")
print(f"  - {c.shape[1]} features (100 base + 100 macro interactions)")
print(f"\nFirst few values:")
print(c.head())

# Returns (Linear model)
r1 = pd.read_csv(f'{data_dir}/r1_1_1.csv')
print(f"\nLinear model returns shape: {r1.shape}")
print(f"Summary statistics:")
print(r1.describe())

# Returns (Nonlinear model)
r2 = pd.read_csv(f'{data_dir}/r2_1_1.csv')
print(f"\nNonlinear model returns shape: {r2.shape}")
print(f"Summary statistics:")
print(r2.describe())

## Part 3: Train Neural Networks (PyTorch)

Now we train 5 different neural network architectures:

1. **NN1**: 1 hidden layer (32 neurons)
2. **NN2**: 2 hidden layers (32 → 16)
3. **NN3**: 3 hidden layers (32 → 16 → 8)
4. **NN4**: 4 hidden layers (32 → 16 → 8 → 4)
5. **NN5**: 5 hidden layers (32 → 16 → 8 → 4 → 2)

### Training Strategy:
- **Ensemble learning**: Train 5 models per architecture with different random seeds
- **Hyperparameter search**: Grid search over learning rates and weight decay
- **Early stopping**: Stop when validation loss doesn't improve for 3 epochs
- **Batch normalization**: After each hidden layer for training stability
- **ReLU activation**: For non-linearity

### Expected Runtime:
- ~4-5 minutes per architecture
- ~20-25 minutes total for all 5 architectures

In [ ]:
from NN_pytorch import MC_NN_pytorch

# Configuration
MC = [1]          # Monte Carlo sample(s) to use
datanum = 100     # Number of characteristics (100 or 200)
horizon = [1]     # Return horizon in months
mo = [1]          # Model specification (1=linear, 2=nonlinear)
redo = 5          # Ensemble size (number of models per architecture)

print("Starting neural network training...")
print(f"Configuration:")
print(f"  Monte Carlo samples: {MC}")
print(f"  Data size: {datanum} characteristics")
print(f"  Horizon: {horizon[0]} month(s)")
print(f"  Model: {'Linear' if mo[0] == 1 else 'Nonlinear'}")
print(f"  Ensemble size: {redo} models per architecture")
print(f"\nThis will train 5 architectures × {redo} ensemble members = {5*redo} total models")
print("\n" + "="*70 + "\n")

# Train all models
MC_NN_pytorch(MC, datanum, horizon, mo, redo)

## Part 4: Analyze Results

Let's examine the performance of each architecture:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load results
results_dir = f'./Simu/Simu_{datanum}/NN{horizon[0]}/'

r2_is = pd.read_csv(f'{results_dir}/r2_is_{mo[0]}_1.csv', header=None).values.flatten()
r2_oos = pd.read_csv(f'{results_dir}/r2_oos_{mo[0]}_1.csv', header=None).values.flatten()

models = ['NN1', 'NN2', 'NN3', 'NN4', 'NN5']

# Create results DataFrame
results_df = pd.DataFrame({
    'Model': models,
    'In-Sample R²': r2_is,
    'Out-of-Sample R²': r2_oos
})

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(results_df.to_string(index=False))
print("="*60)

# Visualize results
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

x = np.arange(len(models))
width = 0.35

bars1 = ax.bar(x - width/2, r2_is, width, label='In-Sample R²', alpha=0.8)
bars2 = ax.bar(x + width/2, r2_oos, width, label='Out-of-Sample R²', alpha=0.8)

ax.set_xlabel('Model Architecture', fontsize=12)
ax.set_ylabel('R²', fontsize=12)
ax.set_title('Neural Network Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Best model
best_oos_idx = np.argmax(r2_oos)
print(f"\n🏆 Best out-of-sample performance: {models[best_oos_idx]}")
print(f"   R² (out-of-sample): {r2_oos[best_oos_idx]:.4f}")

## Part 5: Compare with Nonlinear Model (Optional)

Train on the nonlinear DGP model to see how neural networks handle complex interactions:

In [ ]:
# Train on nonlinear model
mo_nonlinear = [2]

print("Training on NONLINEAR model...")
print("This model includes:")
print("  - Quadratic terms: 2*c[0]^2")
print("  - Interaction terms: 1.5*c[0]*c[1]")
print("  - Threshold effects: 0.6*sign(c*y[2])")
print("\n" + "="*70 + "\n")

MC_NN_pytorch(MC, datanum, horizon, mo_nonlinear, redo)

## Summary

### Key Findings:
1. **Architecture matters**: Deeper networks may capture more complex patterns but can also overfit
2. **Ensemble learning**: Averaging multiple models improves robustness
3. **Regularization**: Weight decay helps prevent overfitting
4. **Early stopping**: Prevents training for too long and overfitting

### PyTorch Advantages:
- More explicit control over model architecture
- Better debugging capabilities
- Clearer computational graph
- More flexible for research

### Next Steps:
- Experiment with different architectures
- Try different regularization techniques
- Test on real asset pricing data
- Compare with other ML methods (random forests, boosting, etc.)